In [105]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from datasets import load_dataset
import torchvision
from torchvision import transforms
import numpy as np


In [106]:
### Hyperparams 
batch_size = 64
num_workers = 4
validation_ratio = 0.2

In [107]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])


In [108]:
path = r'C:\python\datasets_storage'
test_data = torchvision.datasets.CIFAR100(root=path,train=False,download=True, transform=transform)
dataset_train = torchvision.datasets.CIFAR100(root=path,train=True,download=True, transform=transform)

train_data,validation_data=torch.utils.data.random_split(dataset_train,[int((1-validation_ratio)*len(dataset_train)), int((validation_ratio)*len(dataset_train))])
print(len(validation_data))
print(len(train_data))

Files already downloaded and verified
Files already downloaded and verified
10000
40000


In [109]:
train_loader = DataLoader(train_data, batch_size=batch_size, pin_memory=True, shuffle=True)
val_loader = DataLoader(validation_data, batch_size=batch_size, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size,shuffle=True, pin_memory=True)

In [110]:
def check_device():
    if torch.cuda.is_available:
        return torch.device('cuda')
    else:
        return torch.device('cpu')
def move_device(tensor,device):
    if isinstance(tensor,(list,tuple)):    
      return[move_device(element,device) for element in tensor]  
    return tensor.to(device,non_blocking=True)
class DeviceDataloader():
    def __init__(self,dataloader,device):
        self.dataloader = dataloader
        self.device = device
    def __iter__(self):
        # transfer ecach batch and return
        for i in self.dataloader:
           yield move_device(i, self.device)
    def __len__(self):

        return len(self.dataloader)
device = check_device()

train_dl = DeviceDataloader(train_loader, device)
test_dl = DeviceDataloader(test_loader, device)
val_dl  = DeviceDataloader(val_loader, device)
#len(train_dl)

In [111]:
class CNN_model(nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(CNN_model,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(32), # out channels, helps normalize batches and imporves overall better performance
            nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, stride = 1, padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Dropout(dropout_rate)
        )
        self.hidden_layer = nn.Linear(256 * 2 * 2, 512)
        self.output_layer = nn.Linear(512, 100)
    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2(output)
        output = self.conv3(output)
        output = self.conv4(output)
        #flattening 
        output = output.view(-1, 256 * 2 * 2)
        # fully connected layers
        output = self.hidden_layer(output)
        output = self.output_layer(output)
        return output
model = CNN_model()
print(model)

CNN_model(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
  

In [112]:
torch.save(model, 'img_model.pth')

In [113]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
#optimizer = optim.SGD(model.parameters(), lr= 3e-3, momentum=0.9)
optimizer = optim.AdamW(model.parameters(), lr=3e-3)

In [114]:
print(torch.cuda.is_available())
print(torch.version.cuda if torch.cuda.is_available() else "CUDA not found")

True
12.1


In [115]:
print('Starting...')
epochs = 55
model = model.to(device)
def model_train(model, train_loader, criterion, optimizer):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for batch_idx,(images,targets) in enumerate(train_loader):
            optimizer.zero_grad()
            images,targets = images.to(device), targets.to(device)
            outputs = model(images)
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() ### compares predicted lables to true lables and returns True/False
            batch_idx += 1
        
            if batch_idx % 100 == 0:
                print(f'Epoch" {epoch + 1}/{epochs} | Loss:{running_loss} | '
                      f'Loss: {running_loss / (batch_idx+1):.4f} | '
                      f'Acc_train: {100.*correct/total:.2f}% | '
                      f'Batch: {batch_idx / len(train_loader)}')
def model_val(model, val_loader):
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        for images, targets in val_loader:
            with torch.no_grad():
                images,targets = images.to(device), targets.to(device)
                outputs = model(images)
    
                loss = criterion(outputs, targets)
    
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                return val_loss/len(val_loader), 100.* correct/total
        print('Finished!')

Starting...


In [116]:
model_train(model,train_loader, criterion, optimizer)
val_loss, val_acc = model_val(model, val_loader)

Epoch" 1/55 | Loss:492.31095838546753 | Loss: 4.8744 | Acc_train: 2.27% | Batch: 0.16
Epoch" 1/55 | Loss:930.823037147522 | Loss: 4.6310 | Acc_train: 3.09% | Batch: 0.32
Epoch" 1/55 | Loss:1357.230051279068 | Loss: 4.5091 | Acc_train: 3.89% | Batch: 0.48
Epoch" 1/55 | Loss:1773.321139574051 | Loss: 4.4222 | Acc_train: 4.82% | Batch: 0.64
Epoch" 1/55 | Loss:2181.2392127513885 | Loss: 4.3538 | Acc_train: 5.63% | Batch: 0.8
Epoch" 1/55 | Loss:2581.3448598384857 | Loss: 4.2951 | Acc_train: 6.33% | Batch: 0.96
Epoch" 2/55 | Loss:392.73014521598816 | Loss: 3.8884 | Acc_train: 12.73% | Batch: 0.16
Epoch" 2/55 | Loss:776.4895262718201 | Loss: 3.8631 | Acc_train: 13.61% | Batch: 0.32
Epoch" 2/55 | Loss:1158.5371389389038 | Loss: 3.8490 | Acc_train: 13.89% | Batch: 0.48
Epoch" 2/55 | Loss:1537.400114774704 | Loss: 3.8339 | Acc_train: 14.14% | Batch: 0.64
Epoch" 2/55 | Loss:1913.0721576213837 | Loss: 3.8185 | Acc_train: 14.33% | Batch: 0.8
Epoch" 2/55 | Loss:2283.771164894104 | Loss: 3.8000 | Acc

In [117]:
print(f'Val_loss: {val_loss:.4f}, Val_acc: {val_acc}%')

Val_loss: 0.0164, Val_acc: 51.5625%


In [118]:
def model_test(model, test_loader):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    for images, targets in test_loader:
        with torch.no_grad():
            images,targets = images.to(device), targets.to(device)
            outputs = model(images)

            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            total = predicted.size(0)
            correct = predicted.eq(targets).sum().item()
            return test_loss/len(test_loader), 100.*correct/total
test_loss,test_acc = model_test(model, test_loader)
print(f"test loss: {test_loss} | test acc: {test_acc}%")

test loss: 0.016890619970430995 | test acc: 40.625%


In [119]:
# ------------- 5 epochs --------------- lr = 3e-3
#Acc_train: 26.01%. Val_acc: 28.125% test acc: 29.6875% - no weight_decay/ SGD opimizer
#Acc_train: 25.56%  Val_acc: 25.0%   test acc: 35.9375% - AdamW optimizer
#Acc_train: 25.09%  Val_acc: 25.0%   test acc: 32.8125% - AdamW with 1e-2 weight_decay
# ------------- 5 epochs --------------- lr = 3e-4
#Acc_train: 28.55%  Val_acc: 28.125% test acc: 23.4375%
# -------------- 40 epochs -------------- lr = 3e-3
#Acc_train: 41.29%  Val_acc: 56.25%   test acc: 48.4375% - AdamW

In [120]:
new_cifar_data = torchvision.datasets.CIFAR100(root=path, download=True, transform=transform)
new_cifar_loader = DataLoader(new_cifar_data, batch_size=32, pin_memory=4, shuffle=5)

Files already downloaded and verified


In [125]:
correct = 0
x = 15
for i in range(x):
    total = x    
    image, label = new_cifar_data[i]
    image = image.unsqueeze(0)
    model.to('cpu')
    model.eval()
    output = model(image)
    predicted = torch.argmax(output, dim=1).item()
    if  predicted == label:
        correct += 1
    
    print(f'Predicted class: {predicted}/{label}')
accuracy = 100.*correct/total
print(f'correct/total: {correct}/{total}')
print(f'Accuracy:{accuracy}%')

Predicted class: 89/19
Predicted class: 29/29
Predicted class: 0/0
Predicted class: 11/11
Predicted class: 1/1
Predicted class: 22/86
Predicted class: 27/90
Predicted class: 10/28
Predicted class: 71/23
Predicted class: 31/31
Predicted class: 39/39
Predicted class: 52/96
Predicted class: 82/82
Predicted class: 37/17
Predicted class: 71/71
correct/total: 8/15
Accuracy:53.333333333333336%
